In [2]:
import openai
import csv
import pandas as pd
import json
import re
import numpy as np
API_KEY = "your_API_key" 
openai.api_key = API_KEY

## Scenarios & Prompts 
##forest
gain_forest = """Scenario: A vast forest area is under severe threat from deforestation due to urban growth and logging activities, potentially resulting in the removal of 900 trees from a grove of ancient oak trees in the forest. Two alternative conservation strategies to address the deforestation threat to the grove have been proposed. Imagine you are on the conservation committee tasked with deciding on a proposal. Assume that the estimates are as follows:\n- Option A: If Proposal A is adopted, 300 of the trees will be protected.\n- Option B: If Proposal B is adopted, there is a 1/3 probability that all 900 of the trees will be protected, and 2/3 probability that no trees will be protected."""
loss_forest = """Scenario: A vast forest area is under severe threat from deforestation due to urban growth and logging activities, potentially resulting in the removal of 900 trees from a grove of ancient oak trees in the forest. Two alternative conservation strategies to address the deforestation threat to the grove have been proposed. Imagine you are on the conservation committee tasked with deciding on a proposal. Assume that the estimates are as follows:\n- Option A: If Proposal A is adopted, 600 of the trees will be cleared.\n- Option B: If Proposal B is adopted, there is a 1/3 probability that none of the trees will be cleared, and 2/3 probability that all 900 of the trees will be cleared."""

##human
gain_human = """Scenario: A city is facing significant air quality issues due to high levels of pollution, which could lead to 900 premature deaths over the next year due to respiratory illness. Two alternative plans to improve the air quality and reduce pollution have been proposed. Imagine you are a city official on the committee tasked with deciding on a proposal. Assume that the estimates are as follows:\n- Option A: If Proposal A is adopted, 300 people will survive.\n- Option B: If Proposal B is adopted, there is a 1/3 probability that all 900 people will survive, and 2/3 probability that no one will survive."""
loss_human = """Scenario: A city is facing significant air quality issues due to high levels of pollution, which could lead to 900 premature deaths over the next year due to respiratory illness. Two alternative plans to improve the air quality and reduce pollution have been proposed. Imagine you are a city official on the committee tasked with deciding on a proposal. Assume that the estimates are as follows:\n- Option A: If Program A is adopted, 600 people will die.\n- Option B: If Program B is adopted, there is a 1/3 probability that no one will die, and 2/3 probability that all 900 people will die."""

#animal (oil spill) scenario
gain_animal = """Scenario: An oil spill off the coast of a country is anticipated to cause significant environmental damage, and is projected to endanger the lives of 900 marine animals, including sea turtles, seals, and various types of seabirds. Two alternative clean-up efforts to address the oil spill have been proposed. Imagine you are on the clean-up committee tasked with deciding on a proposal. Assume that the estimates are as follows:\n- Option A: If Proposal A is adopted, 300 of the marine animals will survive.\n- Option B: If Proposal B is adopted, there is a 1/3 probability that all 900 of the marine animals will survive, and 2/3 probability that no marine animals will survive."""
loss_animal = """Scenario: An oil spill off the coast of a country is anticipated to cause significant environmental damage, and is projected to endanger the lives of 900 marine animals, including sea turtles, seals, and various types of seabirds. Two alternative clean-up efforts to address the oil spill have been proposed. Imagine you are on the clean-up committee tasked with deciding on a proposal. Assume that the estimates are as follows:\n- Option A: If Proposal A is adopted, 600 of the marine animals will die.\n- Option B: If Proposal B is adopted, there is a 1/3 probability that no marine animals will die, and 2/3 probability that all 900 of the marine animals will die."""


##################################### Comboos for Experiment #####################################

####### System Role ####### 

###Combo 1: Baseline Condition (formerly, combo 8B) ##tested
    # participant 
    # capable of decisions
    # basic instructions -- user messsage 
    ## user_message_baseline1 = user_message_4 
system_message_baseline1 = "You are a participant in a survey. You are capable of making decisions and choices. Provide your response as a JSON object."
user_message_baseline1 = """You are participating in a survey-based study.\nInstructions: read the scenario below and then answer the questions.\n{}\n\nQuestions:\n1. Please choose an option:\n- Proposal A\n- Proposal B\n2. On a scale of 1 (Strong Preference for Proposal A) to 7 (Strong Preference for Proposal B), rate your preference for the option selected:\n- 1: Strong Preference for Proposal A\n- 2: Preference for Proposal A\n- 3: Slight Preference for Proposal A\n- 4: No Preference for Proposal A or B\n- 5: Slight Preference for Proposal B\n- 6: Preference for Proposal B\n- 7: Strong Preference for Proposal B\n3. Briefly describe the rationale for your preference rating:\n\nProvide your response as a JSON object. As an example: '{{"Q1_Response": "Your Proposal Choice", "Q2_Response": "Your Rating", "Q3_Response": "Your Rationale."}}'.\n\nResponse:"""

###Combo 2: Human Participant 
    # (+) human participant -- system messsage 
    # capable of decisions -- system messsage 
    # single JSON -- system message
    # basic instructions -- user messsage  
    # Task Order Instructions via one JSON object -- user messsage 
system_message_hum  = "You are a human participant in a survey. You are capable of making decisions and choices. Provide your response as a JSON object."
user_message = user_message_baseline1

###Combo 3: Adds in Risk Preferences 
    # human participant -- system messsage 
    # capable of decisions -- system messsage 
    # (+) Consider human risk preferenes -- system messsage
    # single JSON -- system message
    # basic instructions -- user messsage  
    # Task Order Instructions via one JSON object -- user messsage 
system_message_risk = "You are a human participant in a survey. You are capable of making decisions and choices. You are responding to a survey with the same risk preferences as a human participant would. Provide your response as a JSON object." 
user_message = user_message_baseline1


###Combo 4: Adds in Risk Preferences to user message
 # human participant -- system messsage 
    # capable of decisions -- system messsage 
    # Consider human risk preferenes -- system messsage
    # single JSON -- system message
    # basic instructions -- user messsage  
    # Task Order Instructions via one JSON object -- user messsage 
    # (+) Consider human risk preferenes -- user messsage

system_message = system_message_risk
user_message_risk = """You are participating in a survey-based study. Respond with the same risk preferences as a human participant would.\nInstructions: read the scenario below and then answer the questions.\n{}\n\nQuestions:\n1. Please choose an option:\n- Proposal A\n- Proposal B\n2. On a scale of 1 (Strong Preference for Proposal A) to 7 (Strong Preference for Proposal B), rate your preference for the option selected:\n- 1: Strong Preference for Proposal A\n- 2: Preference for Proposal A\n- 3: Slight Preference for Proposal A\n- 4: No Preference for Proposal A or B\n- 5: Slight Preference for Proposal B\n- 6: Preference for Proposal B\n- 7: Strong Preference for Proposal B\n3. Briefly describe the rationale for your preference rating:\n\nProvide your response as a JSON object. As an example: '{{"Q1_Response": "Your Proposal Choice", "Q2_Response": "Your Rating", "Q3_Response": "Your Rationale."}}'.\n\nResponse:"""

## Define Functions & Model Call
## Create functions to make prompt scenario dynamics 
def format_scenario(scenario_text):
    return scenario_text

def format_prompt(prompt_text, scenario):
    return prompt_text.format(scenario)

##define scenario and user_message variables 
def generate_response_turbo(sys_prompt, user_prompt, temperature, top_p):
     messages=[{"role": "system", "content" : sys_prompt },
               {"role": "user", "content" : user_prompt }]
     response = openai.ChatCompletion.create(
          #model="gpt-3.5-turbo",
        model="gpt-3.5-turbo",
          messages = messages,
          temperature=temperature,
          max_tokens=200,
          n=1, 
          top_p=top_p
  )
     prompt = sys_prompt + user_prompt
     return [(prompt, response.choices[0].message['content'], temperature, top_p) for choice in response.choices ]

     ## Call Model for Each Scenario
######## Humans ########

## Human Loss 
##humans_loss	37

human_loss_turbo = []

for i in range(37):
    formatted_scenario = format_scenario(loss_human) ##change for each run 
    user_message = format_prompt(user_message, 
                                 formatted_scenario) ##change for each run 
    human_loss_output = generate_response_turbo(system_message, 
                                                user_message, .7, 1) 
    prompt, response, temperature, top_p = human_loss_output[0]  
    frame = 'loss_human'
    model = 'gpt-3.5-turbo'
    test = ##add in test
    num_calls = 37 ## matches to qual exp.  
    human_loss_turbo.append({'prompt': prompt, 'response': response, 'temperature': temperature,'top_p': top_p, 'test': test, 'model': model, 'frame': frame, 'num_calls': num_calls})

## Human Gain 
##humans_gain	30

human_gain_turbo = []

for i in range(30):
    formatted_scenario = format_scenario(gain_human) ##change for each run 
    user_message = format_prompt(user_message, 
                                 formatted_scenario) ##change for each run 
    human_gain_output = generate_response_turbo(system_message, 
                                                user_message, .7, 1) 
    prompt, response, temperature, top_p = human_gain_output[0]  
    frame = 'gain_human' 
    model = 'gpt-3.5-turbo'
    test = 30
    num_calls = 20 ## matches to qual exp.  
    
    human_gain_turbo.append({'prompt': prompt, 'response': response, 'temperature': temperature,'top_p': top_p, 'test': test, 'model': model, 'frame': frame, 'num_calls': num_calls})

######## Animals ########
## Animal Loss 
##animals_loss	37
animal_loss_turbo = []
for i in range(37):
    formatted_scenario = format_scenario(loss_animal)  
    user_message = format_prompt(user_message, ##change for each run 
                                 formatted_scenario) 
    animal_loss_output = generate_response_turbo(system_message, ##change for each run 
                                                user_message, .7, 1) 
    prompt, response, temperature, top_p = animal_loss_output[0]  
    frame = 'loss_animal'  
    model = 'gpt-3.5-turbo'
    test = ##add in test
    num_calls = 37
    animal_loss_turbo.append({'prompt': prompt, 'response': response, 'temperature': temperature,'top_p': top_p, 'test': test, 'model': model, 'frame': frame, 'num_calls': num_calls})

## Animal Gain 
##animals_gain	34 

animal_gain_turbo = []
for i in range(34):
    formatted_scenario = format_scenario(gain_animal) 
    user_message = format_prompt(user_message, ##change for each run 
                                 formatted_scenario) 
    animal_gain_output = generate_response_turbo(system_message, ##change for each run 
                                                user_message, .7, 1) 
    prompt, response, temperature, top_p = animal_gain_output[0]  
    frame = 'gain_animal' 
    model = 'gpt-3.5-turbo'
    test = ##add in test
    num_calls = 34 ## matches to qual exp.  
    animal_gain_turbo.append({'prompt': prompt, 'response': response, 'temperature': temperature,'top_p': top_p, 'test': test, 'model': model, 'frame': frame, 'num_calls': num_calls})


## Forest Gain
##forest_gain	35
forest_gain_turbo = []
for i in range(35):
    formatted_scenario = format_scenario(gain_forest)  
    user_message = format_prompt(user_message, ##change for each run 
                                 formatted_scenario) 
    forest_gain_output = generate_response_turbo(system_message, ##change for each run 
                                                user_message, .7, 1) 
    prompt, response, temperature, top_p = forest_gain_output[0]  
    frame = 'gain_forest'  
    test = ##add in test
    test = ['combo 1A', 'system_message_baseline1', 'user_message_baseline1', 'system role']
    num_calls = 35 ## matches to qual exp.  
    forest_gain_turbo.append({'prompt': prompt, 'response': response, 'temperature': temperature,'top_p': top_p, 'test': test, 'model': model, 'frame': frame, 'num_calls': num_calls})

## Forest Loss
##forest_loss 28
forest_loss_turbo = []
for i in range(28):
    formatted_scenario = format_scenario(loss_forest) 
    user_message = format_prompt(user_message, ##change for each run 
                                 formatted_scenario) 
    forest_loss_output = generate_response_turbo(system_message, ##change for each run 
                                                user_message, .7, 1) 
    prompt, response, temperature, top_p = forest_loss_output[0]  
    frame = 'loss_forest' 
    test = ##add in test
    num_calls = 28 ## matches to qual exp.  
    forest_loss_turbo.append({'prompt': prompt, 'response': response, 'temperature': temperature,'top_p': top_p, 'test': test, 'model': model, 'frame': frame, 'num_calls': num_calls})


## Append CSV File for multi call model version 

with open(csv_file_path, 'a', newline='') as file:
    fieldnames = ['prompt', 'response', 'temperature', 'top_p', 'test', 'model', 'frame', 'num_calls']
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    

    for response_data in human_loss_turbo:
        writer.writerow({
            'prompt': response_data['prompt'],
            'response': response_data['response'],
            'temperature': response_data['temperature'],
            'top_p': response_data['top_p'],
            'test': response_data['test'],     
            'model': response_data['model'],   
            'frame': response_data['frame'],
            'num_calls': response_data['num_calls']

        })

    for response_data in human_gain_turbo:
        writer.writerow({
            'prompt': response_data['prompt'],
            'response': response_data['response'],
            'temperature': response_data['temperature'],
            'top_p': response_data['top_p'],
            'test': response_data['test'],      
            'model': response_data['model'],    
            'frame': response_data['frame'],
            'num_calls': response_data['num_calls']
        })

    for response_data in animal_gain_turbo:
            writer.writerow({
            'prompt': response_data['prompt'],
            'response': response_data['response'],
            'temperature': response_data['temperature'],
            'top_p': response_data['top_p'],
            'test': response_data['test'],      
            'model': response_data['model'],    
            'frame': response_data['frame'],
            'num_calls': response_data['num_calls']
        })
    for response_data in animal_loss_turbo:
        writer.writerow({
            'prompt': response_data['prompt'],
            'response': response_data['response'],
            'temperature': response_data['temperature'],
            'top_p': response_data['top_p'],
            'test': response_data['test'],     
            'model': response_data['model'],   
            'frame': response_data['frame'],
            'num_calls': response_data['num_calls']
        })

    for response_data in forest_gain_turbo:
            writer.writerow({
            'prompt': response_data['prompt'],
            'response': response_data['response'],
            'temperature': response_data['temperature'],
            'top_p': response_data['top_p'],
            'test': response_data['test'],      
            'model': response_data['model'],    
            'frame': response_data['frame'],
            'num_calls': response_data['num_calls']
        })
    for response_data in forest_loss_turbo:
        writer.writerow({
            'prompt': response_data['prompt'],
            'response': response_data['response'],
            'temperature': response_data['temperature'],
            'top_p': response_data['top_p'],
            'test': response_data['test'],     
            'model': response_data['model'],   
            'frame': response_data['frame'],
            'num_calls': response_data['num_calls']
        })

In [3]:
## Load Data 
gpt3_run = csv_file_path
df_gpt3 = pd.read_csv(gpt3_run)

## Cleaning
##info on prompt
df_gpt3[['frame', 'scenario']] = df_gpt3['frame'].str.split('_', n=1, expand=True)
pattern2 = r'Scenario:(.*?)Questions'
df_gpt3['scenario_text'] = df_gpt3['prompt'].str.extract(pattern2,  flags=re.DOTALL)

## Clean json 
# import json 
def clean_json(x):
    x = x.strip("'")  # remove leading/trailing single quotes
    x = x.strip("\n")  # remove leading/trailing newline characters
    x = x.encode('utf-8', 'ignore').decode('utf-8')  # ignore non utf-8 characters
    x = re.sub(r'"\s*"', '","', x)  # replace spaces between quotes with commas
    #replace incorrect JSON keys with correct keys
    for key in ["Q1_Response", "Q2_Response", "Q3_Response"]:
        x = re.sub(f'(?<=[{{,])\s*{key}\s*(?=:)', f' "{key}"', x)
    # make sure quotes 
    x = re.sub(r':\s*([0-9]+)\s*(?=[,}])', r': "\1"', x)

    # cleaning steps
    x = x.replace('Q1_response','Q1_Response')
    x = x.replace('Q2_response','Q2_Response')
    x = x.replace('Q3_response','Q3_Response')
    x = x.replace('it"s', 'it\'s')
    x = x.replace('B"s', "B's")
    x = x.replace("}, {", ",")
    x = x.replace("},{", ",")
    x = x.replace(".}", '."}')
    x = x.replace('"}" }', '"}')

    for proposal in ["Proposal A", "Proposal B"]:
        x = re.sub(f'(?<=:)\s*{proposal}(?=\s*[^"]\w*,)', f' "{proposal}"', x)
    x = re.sub(r'}\s*{', ', ', x)
    if not x.startswith('{'):
        x = '{' + x
    if not x.endswith('}'):
        x = x + '}'
    try:
        x = x.replace('\n', ' ')
        x = json.dumps(json.loads(x))
    except json.JSONDecodeError:
        return x
    return x

df_gpt3['response'] = df_gpt3['response'].apply(clean_json)
responses = df_gpt3.apply(lambda x: pd.Series(json.loads(x['response'])), axis=1, result_type='expand')
df_gpt3 = pd.concat([df_gpt3, responses], axis = 1)
df_gpt3 = df_gpt3.reset_index()
df_gpt3['Q1_Response']  = df_gpt3['Q1_Response'].apply(lambda x: 'Proposal B' if x == 'Option B' else x)
df_gpt3['Q1_Response']  = df_gpt3['Q1_Response'].apply(lambda x: 'Proposal A' if x == 'Option A' else x)

## Save File 
df_gpt3.to_csv('/Users/annaking/Documents/Github/riskychoiceframing_AI/Experiment 1/analysis/df_gpt3_testing_072623.csv')